In [108]:
import pickle

In [109]:
results_path = f'59411.pkl'
reg = pickle.load(open(results_path, 'rb'))

In [112]:
reg.equations_[:10]

,complexity,loss,score,equation,sympy_format,lambda_format
0,1,86.392230,0.000000,c,c,PySRFunction(X=>c)
1,3,34.445393,0.459761,c + a,a + c,PySRFunction(X=>a + c)
2,5,28.011744,0.103376,(a * c) ^ 0.71505386,(a*c)**0.71505386,PySRFunction(X=>(a*c)**0.71505386)
3,6,13.497760,0.730100,sqrt(c * (b * a)),sqrt(a*b*c),PySRFunction(X=>sqrt(a*b*c))
4,7,5.194586,0.954907,((c * b) * a) * 0.053678405,0.053678405*a*b*c,PySRFunction(X=>0.053678405*a*b*c)
5,9,4.168063,0.110083,(c * (a * (b * 0.048061028))) - -1.6854905,0.048061028*a*b*c + 1.6854905,PySRFunction(X=>0.048061028*a*b*c + 1.6854905)
6,11,4.125393,0.005145,((b * (a * (0.05816862 * c))) + 1.4348089) ^ 0...,1.40907795815168*(0.0405410225710197*a*b*c + 1...,PySRFunction(X=>1.40907795815168*(0.0405410225...
7,12,3.921815,0.050607,(b * (a * (0.22453886 * (sqrt(c) - 1.1047957))...,a*b*(0.22453886*sqrt(c) - 0.248069567010902) +...,PySRFunction(X=>a*b*(0.22453886*sqrt(c) - 0.24...
8,14,3.916114,0.000727,((b * ((0.22453886 * (sqrt(c) - 1.1047957)) * ...,a*b*(0.22453886*sqrt(c) - 0.248069567010902) +...,PySRFunction(X=>a*b*(0.22453886*sqrt(c) - 0.24...
9,15,3.888544,0.007065,(((a * (0.05052264 * c)) * b) - -1.8437126) - ...,0.05052264*a*b*c - (0.1144077*b)**b + 1.8437126,PySRFunction(X=>0.05052264*a*b*c - (0.1144077*...


In [114]:
import re

def replace_constants(expression: str) -> tuple:
    constants = {}
    constant_index = 1

    # Find all constants in the expression
    def replace(match):
        nonlocal constant_index
        value = match.group()
        key = f"C{constant_index}"
        constants[key] = float(value)
        constant_index += 1
        return key

    # Regular expression to find floating point and integer constants
    modified_expression = re.sub(r'(?<![a-zA-Z])[-+]?\d*\.\d+|\d+', replace, expression)

    return modified_expression, constants

In [115]:
from sympy import sympify, simplify

def simplify_expression(expression: str) -> str:
    # Convert the string expression to a SymPy expression
    sympy_expr = sympify(expression)
    # Simplify the expression
    simplified_expr = simplify(sympy_expr)
    # Return the simplified expression as a string
    return str(simplified_expr)

In [116]:
from sympy import sympify, Function, Mul, Add, Pow, sqrt

def expression_to_lambda_format(expression: str) -> str:
    sympy_expr = sympify(expression)

    def convert(expr):
        if isinstance(expr, Function):  # Handle functions like sqrt, sin, cos, etc.
            func_name = expr.func.__name__
            args = " ".join(convert(arg) for arg in expr.args)
            return f"({func_name} {args})"
        elif isinstance(expr, Mul):  # Handle multiplication
            args = " ".join(convert(arg) for arg in expr.args)
            return f"(* {args})"
        elif isinstance(expr, Add):  # Handle addition
            args = " ".join(convert(arg) for arg in expr.args)
            return f"(+ {args})"
        elif isinstance(expr, Pow):  # Handle exponentiation
            base, exp = expr.args
            return f"(pow {convert(base)} {convert(exp)})"
        elif expr.is_Number or expr.is_Symbol:  # Base case for numbers or variables
            return str(expr)
        else:
            raise ValueError(f"Unsupported expression type: {type(expr)}")

    return convert(sympy_expr)

In [125]:
# Example:
eq = reg.equations_['equation'][6]
print('Equation:', eq)
cleaned_expr, d = replace_constants(eq)
print('Coefficients removed (so simpy does not mess with them):', cleaned_expr)
print('Constant dict:', d)
simp_expr = simplify_expression(cleaned_expr)
print('Simplified expr:', simp_expr)
lamb_expr = expression_to_lambda_format(simp_expr)
print('Lambdified expr:', lamb_expr)


Equation: ((b * (a * (0.05816862 * c))) + 1.4348089) ^ 0.94987667
Coefficients removed (so simpy does not mess with them): ((b * (a * (C1 * c))) + C2) ^ C3
Constant dict: {'C1': 0.05816862, 'C2': 1.4348089, 'C3': 0.94987667}
Simplified expr: (C1*a*b*c + C2)**C3
Lambdified expr: (pow (+ C2 (* C1 a b c)) C3)


In [132]:
for i, eq in enumerate(reg.equations_['equation']):
    print(f'Equation {i}:', eq)
    cleaned_expr, d = replace_constants(eq)
    # print('Coefficients removed:', cleaned_expr)
    # print('Constant dict:', d)
    # simp_expr = simplify_expression(cleaned_expr)
    # print('Simplified expr:', simp_expr)
    lamb_expr = expression_to_lambda_format(simp_expr)
    print('Lambdified expr:', lamb_expr)
    print()

Equation 0: c
Lambdified expr: (pow (* (pow C10 -1) (+ C8 (pow (+ (pow (* (pow C1 -1) (pow C3 -1) (+ (* C1 C2) (* C3 a b c))) 1/2) (* -1 C4 C5)) C6) (* -1 C9) (* -1 a) (* -1 b) (* -1 c) (* -1 (pow C7 1/2)))) C11)

Equation 1: c + a
Lambdified expr: (pow (* (pow C10 -1) (+ C8 (pow (+ (pow (* (pow C1 -1) (pow C3 -1) (+ (* C1 C2) (* C3 a b c))) 1/2) (* -1 C4 C5)) C6) (* -1 C9) (* -1 a) (* -1 b) (* -1 c) (* -1 (pow C7 1/2)))) C11)

Equation 2: (a * c) ^ 0.71505386
Lambdified expr: (pow (* (pow C10 -1) (+ C8 (pow (+ (pow (* (pow C1 -1) (pow C3 -1) (+ (* C1 C2) (* C3 a b c))) 1/2) (* -1 C4 C5)) C6) (* -1 C9) (* -1 a) (* -1 b) (* -1 c) (* -1 (pow C7 1/2)))) C11)

Equation 3: sqrt(c * (b * a))
Lambdified expr: (pow (* (pow C10 -1) (+ C8 (pow (+ (pow (* (pow C1 -1) (pow C3 -1) (+ (* C1 C2) (* C3 a b c))) 1/2) (* -1 C4 C5)) C6) (* -1 C9) (* -1 a) (* -1 b) (* -1 c) (* -1 (pow C7 1/2)))) C11)

Equation 4: ((c * b) * a) * 0.053678405
Lambdified expr: (pow (* (pow C10 -1) (+ C8 (pow (+ (pow (* (pow 

In [61]:
def stitchify_expr(expression):
    cleaned_expr, d = replace_constants(expression)
    simp_expr = simplify_expression(cleaned_expr)
    lamb_expr = expression_to_lambda_format(simp_expr)
    return lamb_expr

In [62]:
stitchified_exprs = [stitchify_expr(eq) for eq in reg.equations_['equation']]

In [131]:
for expr in stitchified_exprs:
    print(expr)


c
(+ a c)
(pow (* a c) C1)
(pow (* a b c) 1/2)
(* C1 a b c)
(+ (* -1 C2) (* C1 a b c))
(pow (+ C2 (* C1 a b c)) C3)
(+ (* -1 C3) (* -1 C1 a b (+ C2 (* -1 (pow c 1/2)))))
(+ C3 (* -1 C4) (* -1 C1 a b (+ C2 (* -1 (pow c 1/2)))))
(+ (* -1 C2) (* -1 (pow (* C3 b) b)) (* C1 a b c))
(+ (* -1 C3) (* -1 (pow (* C4 b) b)) (* C1 a c (+ C2 b)))
(+ (* -1 C4) (* -1 (pow (* C5 b) b)) (* (+ C2 (* C1 a c)) (+ C3 b)))
(+ C1 C4 (pow (* a b c) (* 1/2 C3)) (* -1 C2) (* -1 a) (* -1 b) (* -1 c))
(+ C4 (pow (+ (pow (* a b c (pow C1 -1)) 1/2) (* -1 C2)) C3) (* -1 C5) (* -1 a) (* -1 b) (* -1 c))
(* (pow C5 1/2) (+ C4 (pow (+ (pow (* a b c (pow C1 -1)) 1/2) (* -1 C2)) C3) (* -1 a) (* -1 b) (* -1 c)))
(pow (* (pow C5 -1) (+ C4 (pow (+ (pow (* a b c (pow C1 -1)) 1/2) (* -1 C2)) C3) (* -1 a) (* -1 b) (* -1 c))) C6)
(pow (* (pow C6 -1) (+ C5 (pow (+ (pow (* (pow C2 -1) (+ C1 (* a b c))) 1/2) (* -1 C3)) C4) (* -1 a) (* -1 b) (* -1 c))) C7)
(pow (* (pow C7 -1) (+ C5 C6 (pow (+ (pow (* (pow C2 -1) (+ C1 (* a b c))) 1/

In [142]:
from stitch_core import compress
res = compress(stitchified_exprs, iterations=1, max_arity=1)

In [143]:
res.abstractions

[fn_0(#0) := (#0 (* -1 a) (* -1 b) (* -1 c))]

In [135]:
res.rewritten

['c',
 '(+ a c)',
 '(pow (* a c) C1)',
 '(pow (* a b c) 1/2)',
 '(* C1 a b c)',
 '(+ (* -1 C2) (* C1 a b c))',
 '(pow (+ C2 (* C1 a b c)) C3)',
 '(+ (* -1 C3) (* -1 C1 a b (+ C2 (* -1 (pow c 1/2)))))',
 '(+ C3 (* -1 C4) (* -1 C1 a b (+ C2 (* -1 (pow c 1/2)))))',
 '(+ (* -1 C2) (* -1 (pow (* C3 b) b)) (* C1 a b c))',
 '(+ (* -1 C3) (* -1 (pow (* C4 b) b)) (* C1 a c (+ C2 b)))',
 '(+ (* -1 C4) (* -1 (pow (* C5 b) b)) (* (+ C2 (* C1 a c)) (+ C3 b)))',
 '(fn_0 (+ C1 C4 (pow (* a b c) (* 1/2 C3)) (* -1 C2)))',
 '(fn_0 (+ C4 (pow (+ (pow (* a b c (pow C1 -1)) 1/2) (* -1 C2)) C3) (* -1 C5)))',
 '(* (pow C5 1/2) (fn_0 (+ C4 (pow (+ (pow (* a b c (pow C1 -1)) 1/2) (* -1 C2)) C3))))',
 '(pow (* (pow C5 -1) (fn_0 (+ C4 (pow (+ (pow (* a b c (pow C1 -1)) 1/2) (* -1 C2)) C3)))) C6)',
 '(pow (* (pow C6 -1) (fn_0 (+ C5 (pow (+ (pow (* (pow C2 -1) (+ C1 (* a b c))) 1/2) (* -1 C3)) C4)))) C7)',
 '(pow (* (pow C7 -1) (fn_0 (+ C5 C6 (pow (+ (pow (* (pow C2 -1) (+ C1 (* a b c))) 1/2) (* -1 C3)) C4)))) C8)

In [144]:
res = compress(stitchified_exprs, iterations=10, max_arity=0)

In [145]:
res.abstractions

[fn_0() := (* -1),
 fn_1() := (+ C4 (pow (+ (pow (* a b c (pow C1 -1)) 1/2) (fn_0 C2)) C3)),
 fn_2() := (pow (+ (pow (* (pow C1 -1) (+ (* C1 C2) (* a b c))) 1/2) (fn_0 C3 C4)) C5),
 fn_3() := (pow (+ (pow (* (pow C2 -1) (+ C1 (* a b c))) 1/2) (fn_0 C3)) C4),
 fn_4() := (* C1 a b c),
 fn_5() := (fn_0 C1 a b (+ C2 (fn_0 (pow c 1/2)))),
 fn_6() := (fn_0 a),
 fn_7() := (fn_0 b),
 fn_8() := (fn_0 c),
 fn_9() := (* a b c)]